In [49]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb

# Load data
data = pd.read_csv('encode_data.csv')  

data

,Quận,Loại nhà,Diện tích (m²),Mức giá (tỷ),Số phòng ngủ,Số toilet,Loại Nhà (mã hóa),Quận (mã hóa),Khoảng giá,Mức giá (triệu/m²)
0,Gia Lâm,Căn hộ chung cư,55.0,2.07,2,1,0,4,1,37.636364
1,Nam Từ Liêm,Căn hộ chung cư,140.0,9.50,4,3,0,12,1,67.857143
2,Hà Đông,Căn hộ chung cư,98.0,3.19,3,2,0,9,1,32.551020
3,Nam Từ Liêm,Căn hộ chung cư,59.0,2.90,2,2,0,12,1,49.152542
4,Hà Đông,Căn hộ chung cư,92.0,3.50,2,2,0,9,1,38.043478
...,...,...,...,...,...,...,...,...,...,...
7887,Bắc Từ Liêm,Căn hộ chung cư,98.0,6.35,3,2,0,1,1,64.795918
7888,Hà Đông,Nhà mặt phố,225.0,66.00,6,8,1,9,3,293.333333
7889,Nam Từ Liêm,Nhà riêng,40.0,4.00,4,4,2,12,1,100.000000
7890,Gia Lâm,Căn hộ chung cư,70.0,2.45,2,2,0,4,1,35.000000


In [50]:
X = data[['Quận (mã hóa)', 'Loại Nhà (mã hóa)', 'Diện tích (m²)', 'Số phòng ngủ', 'Số toilet', 'Khoảng giá', 'Mức giá (triệu/m²)']]
y = data['Mức giá (tỷ)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [51]:
lr = LinearRegression()

In [52]:
lr.fit(X_train, y_train)

LinearRegression()

In [53]:
y_pred_lr = lr.predict(X_test)

mean_squared_error(y_test, y_pred_lr)

70.42452393670399

In [54]:
RFR = RandomForestRegressor(random_state=13)

In [55]:
param_grid_RFR = {
    'max_depth': [12, 15, 18],
    'n_estimators': [200, 400, 500],
    'min_samples_split': [9, 12, 15]
}

In [56]:
rfr_cv = GridSearchCV(RFR, param_grid_RFR, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

rfr_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=13), n_jobs=-1,
             param_grid={'max_depth': [12, 15, 18],
                         'min_samples_split': [9, 12, 15],
                         'n_estimators': [200, 400, 500]},
             scoring='neg_mean_squared_error')

In [57]:
results = rfr_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng truy cập
df_results = pd.DataFrame(results)

# In các tham số và best_score_ cho mỗi cấu hình tham số
for index, row in df_results.iterrows():
    params = row['params']
    mean_test_score = row['mean_test_score']
    std_test_score = row['std_test_score']
    rmse_score = np.sqrt(-mean_test_score)
    print(f"Parameters: {params}, RMSE: {rmse_score:.4f}, Std: {std_test_score:.4f}")

# In ra best_score_ tổng thể
best_rmse = np.sqrt(-1 * rfr_cv.best_score_)
print(f"Best RMSE: {best_rmse:.4f}")

Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 200}, RMSE: 6.2923, Std: 31.3631
Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 400}, RMSE: 6.2816, Std: 30.3405
Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 500}, RMSE: 6.2981, Std: 30.3837
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 200}, RMSE: 6.9436, Std: 33.7743
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 400}, RMSE: 6.9706, Std: 33.1899
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 500}, RMSE: 6.9851, Std: 33.2212
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 200}, RMSE: 7.8542, Std: 37.7069
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 400}, RMSE: 7.8568, Std: 36.8098
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 500}, RMSE: 7.8785, Std: 36.7732
Parameters: {'max_depth': 15, 'min_samples_split': 9, 'n_estimators

In [58]:
rfr_cv.best_params_

{'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 400}

In [59]:
ridge = Ridge()

In [60]:
param_grid_ridge = {
    'alpha': [10, 12, 14, 16, 18],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
}

In [61]:
ridge_cv = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [62]:
ridge_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [10, 12, 14, 16, 18],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag']},
             scoring='neg_mean_squared_error')

In [64]:
# Lấy kết quả của mỗi lần thử
results = ridge_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng hiển thị
results_df = pd.DataFrame(results)

# In ra các cột quan trọng
print(results_df[['mean_test_score', 'std_test_score', 'param_alpha', 'param_solver']])

# In ra mô hình tốt nhất và các tham số tương ứng
print(f"Best parameters found: {ridge_cv.best_params_}")
print(f"Best mean squared error: {np.sqrt(-ridge_cv.best_score_):.4f}")

    mean_test_score  std_test_score param_alpha param_solver
0       -123.583779       36.630259          10         auto
1       -123.583779       36.630259          10          svd
2       -123.583779       36.630259          10     cholesky
3       -123.572141       36.643722          10         lsqr
4       -123.571509       36.644454          10    sparse_cg
5       -126.512701       40.906456          10          sag
6       -123.582032       36.653003          12         auto
7       -123.582032       36.653003          12          svd
8       -123.582032       36.653003          12     cholesky
9       -123.570448       36.666399          12         lsqr
10      -123.569689       36.667277          12    sparse_cg
11      -126.512765       40.907332          12          sag
12      -123.580452       36.675572          14         auto
13      -123.580452       36.675572          14          svd
14      -123.580452       36.675572          14     cholesky
15      -123.568922     

In [65]:
GBR = GradientBoostingRegressor()

In [66]:
param_grid_GBR = {
    'max_depth': [12, 15],
    'n_estimators': [100, 200],
    'min_samples_leaf': [10, 15],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_features': [0.01, 0.1]
}

In [67]:
GBR_cv = GridSearchCV(GBR, param_grid_GBR, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [68]:
GBR_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [12, 15], 'max_features': [0.01, 0.1],
                         'min_samples_leaf': [10, 15],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

In [69]:
# Lấy kết quả của mỗi lần thử
results = GBR_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng hiển thị
results_df = pd.DataFrame(results)

# In ra các cột quan trọng
print(results_df[['mean_test_score', 'std_test_score', 'param_max_depth', 'param_n_estimators', 
                  'param_min_samples_leaf', 'param_learning_rate', 'param_max_features']])

# In ra mô hình tốt nhất và các tham số tương ứng
print(f"Best parameters found: {GBR_cv.best_params_}")
print(f"Best RMSE: {np.sqrt(-GBR_cv.best_score_):.4f}")

    mean_test_score  std_test_score param_max_depth param_n_estimators  \
0       -441.216087      120.576507              12                100   
1       -386.946963      112.598040              12                200   
2       -443.818835      120.943619              12                100   
3       -391.861120      114.055381              12                200   
4       -441.300958      120.428758              12                100   
5       -386.690472      112.956934              12                200   
6       -443.935606      121.612360              12                100   
7       -392.014796      114.171720              12                200   
8       -441.375269      120.499757              15                100   
9       -386.936037      113.102094              15                200   
10      -444.067652      121.424640              15                100   
11      -391.879706      114.361567              15                200   
12      -441.012779      119.787042   

In [89]:
# Tạo mô hình VotingRegressor kết hợp cả hai mô hình GradientBoostingRegressor và RandomForestRegressor
voting_regressor = VotingRegressor(estimators=[('gbr', GBR_cv.best_estimator_), ('rfr', rfr_cv.best_estimator_)],
                                   weights=[1,3])

In [90]:
voting_regressor.fit(X_train, y_train)

VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(max_depth=15,
                                                       max_features=0.01,
                                                       min_samples_leaf=10,
                                                       n_estimators=200)),
                            ('rfr',
                             RandomForestRegressor(max_depth=12,
                                                   min_samples_split=9,
                                                   n_estimators=400,
                                                   random_state=13))],
                weights=[1, 3])

In [91]:
# Dự đoán trên tập kiểm tra
y_pred = voting_regressor.predict(X_test)

In [92]:
# Tính toán lỗi trung bình bình phương
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error of Voting Regressor: {rmse:.4f}')

Root Mean Squared Error of Voting Regressor: 3.1054


In [94]:
from sklearn.ensemble import AdaBoostRegressor

In [95]:
ada_boost_gbr = AdaBoostRegressor(base_estimator=GBR_cv.best_estimator_, n_estimators=50, random_state=42)

In [97]:
ada_boost_gbr.fit(X_train, y_train)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostRegressor(base_estimator=GradientBoostingRegressor(max_depth=15,
                                                           max_features=0.01,
                                                           min_samples_leaf=10,
                                                           n_estimators=200),
                  random_state=42)

In [98]:
# Dự đoán trên tập kiểm tra
y_pred_ada_boost_gbr = ada_boost_gbr.predict(X_test)

# Tính toán lỗi trung bình bình phương
rmse_ada_boost_gbr = np.sqrt(mean_squared_error(y_test, y_pred_ada_boost_gbr))
print(f'Root Mean Squared Error of AdaBoost with GradientBoostingRegressor: {rmse_ada_boost_gbr:.4f}')


Root Mean Squared Error of AdaBoost with GradientBoostingRegressor: 4.2578
